# L4a: Kernel Functions and Kernel Machines
In this lecture, we'll take a theoretical detour and discuss [positive definite kernel functions](https://en.wikipedia.org/wiki/Positive-definite_kernel) and introduce the ideas behind our first [kernel machine](https://en.wikipedia.org/wiki/Kernel_method), namely [kernel regression](https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote14.html). The key ideas of this lecture include:
* __Positive definite kernel functions__: A positive definite kernel function $k:\mathbb{R}^{\star}\times\mathbb{R}^{\star}\to\mathbb{R}$ is a function that takes two vector arguments and returns a scalar that is in some sense a _similarity_ measure of the two input vectors. A positive definite kernel function produces a kernel matrix $\mathbf{K}$ that is positive (semi)definite.
* __Kernel machines__: A kernel machine is a class of machine learning algorithms that uses kernel functions to implicitly transform input data into a high-dimensional feature space, enabling the solution of non-linear problems using linear classifiers without explicitly computing the coordinates in that space.
* __Kernel regression__: Kernel regression is a technique that uses kernel functions to estimate (potentially) non-linear relationships between variables by assigning weights to data points based on their proximity to a point of interest, allowing for flexible modeling without assuming a specific functional form.

Lecture notes can be found: [here!](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-4/L4a/docs/Notes.pdf)

## Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem.

In [3]:
include("Include.jl");

### Data
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) from `01-03-2014` until `02-07-2025`, along with data for a few exchange-traded funds and volatility products during that time. We load the `orignal_dataset` by calling the `MyMarketDataSet()` function:

In [5]:
original_dataset = MyMarketDataSet() |> x-> x["dataset"];


__Clean the data__: Not all tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquisition or de-listing events. Let's collect only those tickers with the maximum number of trading days.

* First, let's compute the number of records for a company that we know has a maximum value, e.g., `AAPL`, and save that value in the `maximum_number_trading_days` variable:

In [7]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow;

Now, lets iterate through our data and collect only those tickers that have `maximum_number_trading_days` records. Save that data in the `dataset::Dict{String,DataFrame}` variable:

In [9]:
dataset = let

    dataset = Dict{String,DataFrame}();
    for (ticker,data) ∈ original_dataset
        if (nrow(data) == maximum_number_trading_days)
            dataset[ticker] = data;
        end
    end
    dataset
end;

Let's get a list of firms in the cleaned up `dataset` and save it in the `all_tickers` array. We sort the firms alphabetically from `A` to `Z`:

In [11]:
list_of_all_tickers = keys(dataset) |> collect |> sort;

Compute the expected (annualized) excess log growth rate by passing the `dataset` and the entire list of firms we have in the dataset to the [log_growth_matrix(...) method](src/Compute.jl). The log growth rate between time period $j-1$ to $j$, e.g., yesterday to today is defined as:
$$
\begin{equation}
\mu_{j,j-1} = \left(\frac{1}{\Delta{t}}\right)\ln\left(\frac{S_{j}}{S_{j-1}}\right)
\end{equation}
$$
where $\Delta{t}$ denotes the period time step, and $S_{j}$ denote share price in period $j$.
* The log growth rates are stored in the `D::Array{Float64,2}` variable, a $T-1\times{N}$ array of log return values. Each row of the `D` matrix corresponds to a time value, while each column corresponds to a firm

In [13]:
D = let

    # setup some constants -
    Δt = (1/252); # 1-trading day in units of years
    risk_free_rate = 0.0415; # inferred cc risk-free rate

    # compute
    μ = log_growth_matrix(dataset, list_of_all_tickers, Δt = Δt, 
        risk_free_rate = risk_free_rate);

    # return to caller
    μ
end

2791×424 Matrix{Float64}:
 -0.919054     6.23955    -2.91247    …   -0.796891    0.204394  -1.04677
  2.77476      1.02999     1.35089         2.09682    -0.84429    0.944968
  3.27155      0.814097   -0.036132        0.068377    1.1495    -2.62294
  0.604925    17.2184      1.65065         0.233216    3.1178    -0.409728
  1.77459      2.53811     3.27774         0.580177   -2.2102     4.36159
  0.57233     -4.00534    -0.83428    …   -0.904239   -1.95127   -3.15774
  2.81921     -0.525251    4.80423         1.7242     -1.81835   -1.1311
  2.00521      0.972004    1.86659         1.63447     4.40834   -0.179319
  1.27139      1.63263     0.0657592      -1.54858    -2.17846    1.39634
  1.17866      6.08807     0.891078       -1.57352     2.83634   -1.47776
 -0.479168     4.82859     0.96624    …   -0.362761    9.46677   -3.05023
  1.32131      3.57167    -2.38926         0.669113    4.48073    0.299031
 -4.78054      1.34435    -3.05774        -2.19395    -6.69057    1.36462
  ⋮      

Next, let's [z-score center](https://en.wikipedia.org/wiki/Feature_scaling) the continous feature data. In [z-score feature scaling](https://en.wikipedia.org/wiki/Feature_scaling), we subtract off the mean of each feature and then divide by the standard deviation, i.e., $x^{\prime} = (x - \mu)/\sigma$ where $x$ is the unscaled data, and $x^{\prime}$ is the scaled data. Under this scaling regime, $x^{\prime}\leq{0}$ will be values that are less than or equal to the mean value $\mu$, while $x^{\prime}>0$ indicate values that are greater than the mean.

We save the z-score centered growth data data in the `D̄::Array{Float64,2}` variable:

In [15]:
D̄ = let

    # setup -
    number_of_examples = size(D,1);

    D̄ = copy(D);
    for j ∈ eachindex(list_of_all_tickers)
        μ = mean(D[:,j]); # compute the mean
        σ = std(D[:,j]); # compute std

        # rescale -
        for k ∈ 1:number_of_examples
            D̄[k,j] = (D[k,j] - μ)/σ;
        end
    end
    
    D̄
end

2791×424 Matrix{Float64}:
 -0.270236    0.875241   -0.51285    …  -0.172603    0.0340086  -0.326599
  0.735235    0.153683    0.270667       0.365351   -0.156963    0.235764
  0.870466    0.123779    0.0157621     -0.0117454   0.206118   -0.771629
  0.144598    2.39589     0.325758       0.018899    0.564557   -0.146732
  0.462985    0.362567    0.624783       0.0834006  -0.405704    1.20044
  0.135725   -0.543745   -0.130921   …  -0.192559   -0.35855    -0.922629
  0.747335   -0.0617295   0.905321       0.29608    -0.334345   -0.35041
  0.52576     0.145651    0.365443       0.279399    0.799572   -0.0816769
  0.326013    0.237151    0.0344876     -0.312346   -0.399924    0.363208
  0.30077     0.854261    0.186164      -0.316981    0.513302   -0.448288
 -0.150497    0.679814    0.199977   …  -0.0918959   1.72074    -0.892273
  0.339601    0.505721   -0.416694       0.0999343   0.812754    0.0533844
 -1.32135     0.197223   -0.539546      -0.432322   -1.22161     0.354251
  ⋮         

### Constants

In [53]:
γ = 0.0197; # Length scale for the RBF kernel. I estimated this from another source, don't mess with it!

## Kernel Functions
Kernel functions in machine learning are mathematical tools that enable algorithms to operate in high-dimensional spaces without explicitly computing the coordinates in those spaces. _Huh??_

* __What are kernel functions__? A kernel function $k:\mathbb{R}^{\star}\times\mathbb{R}^{\star}\to\mathbb{R}$ takes a pair of vectors $\mathbf{v}_i\in\mathbb{R}^{\star}$ and $\mathbf{v}_j\in\mathbb{R}^{\star}$ as arguments, 
e.g., a pair of feature vectors, a feature vector and a parameter vector, or any two vectors of compatible size 
, computes a scalar value that represents the similarity (in some sense) between the two vector arguments.
* __Common kernel functions__: Common kernel functions include the linear kernel, which is the dot product between two vectors $k(\mathbf{z}_i, \mathbf{z}_j) = \mathbf{z}_i^{\top}\mathbf{z}_j$, the polynomial kernel is defined as: $k_{d}(\mathbf{z}_i, \mathbf{z}_j) = (1+\mathbf{z}_i^{\top}\mathbf{z}_j)^d$ and the radial basis function (RBF) kernel $k_{\gamma}(\mathbf{z}_i, \mathbf{z}_j) = \exp(-\gamma \lVert\mathbf{z}_i - \mathbf{z}_j\rVert_{2}^2)$ where $\gamma$ is a scaling factor, and $\lVert\cdot\rVert^{2}_{2}$ is the squared Euclidean norm
* __What do kernel functions do__? Kernel functions are similarity measures, i.e., they quantify the similarity between pairs of points in _some_ high-dimensional space. For example, we could use a kernel function to measure the similarity between augmented feature vectors $\hat{\mathbf{x}}_{i}$ and $\hat{\mathbf{x}}_{j}$, or we could construct higher dimensional versions of the features, for example, by including $x^{2}, x^{3}, \dots,$ terms in addition the original features. Increasing the dimension is [sometimes referred to as feature engineering](https://en.wikipedia.org/wiki/Feature_engineering), i.e., transforming raw data into a more effective set of features.

Let's look at a couple of example kernels that are [exported by the `KernelFunctions.jl` package](https://github.com/JuliaGaussianProcesses/KernelFunctions.jl) and apply them to our financial dataset.

* __Experiment__: If two tickers are similar industries, e.g., `NVDA` and `AMD`, or 'GS' and `JPM`, then they should have a high similarity score. However, if tickers can be anticorrelated, e.g., `SPY` or `QQQ` and `GLD`. Anticorrelated firms will have low similarity scores.

In [19]:
similarity_score, tᵢ, tⱼ = let

    # Specify two tickers that we are going to compare -
    tᵢ,tⱼ = "NVDA", "WMT"
    i = findfirst(s-> s == tᵢ, list_of_all_tickers); # returns index of ticker 1
    j = findfirst(s-> s == tⱼ, list_of_all_tickers); # returns index of ticker 2

    # get z-score scaled return vectors for (i,j)
    μᵢ,μⱼ = D̄[:,i],D̄[:,j];
    
    # setup the kernel -
    # k = LinearKernel(); # inner product kernel
    k = SqExponentialKernel() ∘ ScaleTransform(γ) # RBF kernel w/scaling parameter γ

    # compute the similarity -
    score = k(μᵢ,μⱼ)

    # return -
    score, tᵢ, tⱼ
end;

In [20]:
println("Tickers: ($(tᵢ),$(tⱼ)) have similarity: $(similarity_score)")

Tickers: (NVDA,WMT) have similarity: 0.3994080617533056


## Can any function be a kernel function?
No! Not all functions can be kernel functions; there are some rules. 
* __Rules for a valid kernel function__: A function $k:\mathbb{R}^{\star}\times\mathbb{R}^{\star}\to\mathbb{R}$ is a _valid kernel function_ if and only if the kernel matrix $\mathbf{K}\in\mathbb{R}^{m\times{m}}$ is positive semidefinite for all possible choices of the data vectors $\mathbf{v}_i$, where $K_{ij} = k(\mathbf{v}_i, \mathbf{v}_j)$.
This is equivalent to saying that all eigenvalues of the kernel matrix $\mathbf{K}$ are non-negative (and real).
Further, for any real valued vector $\mathbf{x}$, the Kernel matrix $\mathbf{K}$ must satisfy $\mathbf{x}^{\top}\mathbf{K}\mathbf{x} \geq 0$. Finally, when the kernel function is an (untransformed) inner product, the Kernel matrix is equal to [the Gram matrix](https://en.wikipedia.org/wiki/Gram_matrix).

* __What is the Gram matrix__? For a set of $n$-vectors, $\mathbf{v}_{1},\mathbf{v}_{2},\dots,\mathbf{v}_{m}$, [the Gram matrix](https://en.wikipedia.org/wiki/Gram_matrix) $\mathbf{K}$ is a $m\times{m}$ matrix with elements  $K_{ij}=\left<\mathbf{v}_{i},\mathbf{v}_{j}\right>$, where $\left<\star,\star\right>$ denotes an inner product. If the vectors $\mathbf{v}_{1},\mathbf{v}_{2},\dots,\mathbf{v}_{m}$ are the _columns_ of the matrix $\mathbf{X}$, then [the Gram matrix](https://en.wikipedia.org/wiki/Gram_matrix) is given by $\mathbf{K} = \mathbf{X}^{\top}\mathbf{X}$ (assuming all the entries in $\mathbf{X}$ are real).

Let's check these properties using our financial data. Let's compute [the Gram matrix](https://en.wikipedia.org/wiki/Gram_matrix) $\mathbf{K}$, and then compute its eigendecomposition [using the `eigen(...)` method exported by the `LinearAlgebra.jl` package](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen).

First, let's build a Kernel matrix $\mathbf{K}$ by computing the $K_{ij} = k(\mathbf{v}_{i},\mathbf{v}_{j})$ for all firms in our dataset (the columns). Save this matrix in the `K₁::Array{Float64,2}` variable.

In [22]:
K₁ = let

    k = LinearKernel(); # inner product kernel
    # k = SqExponentialKernel() ∘ ScaleTransform(γ) # RBF kernel w/scaling parameter γ
    # k(x,y) = norm(x-y).^2; # some rando function that I make up
    X = D̄; # we are going to look at the scaled return data
    number_of_firms = size(D̄,2); # number of columns in the dataset
    K = zeros(number_of_firms, number_of_firms);

    for i ∈ 1:number_of_firms
        vᵢ = X[:,i];
        for j ∈ 1:number_of_firms
            vⱼ = X[:,j];
            K[i,j] = k(vᵢ,vⱼ);
        end
    end

    K
end

424×424 Matrix{Float64}:
 2790.0     853.347   742.766  1162.49   …  1261.86    977.207  1430.76
  853.347  2790.0     769.093   811.779      924.723  1214.06    764.885
  742.766   769.093  2790.0     568.675      738.262   887.252   723.949
 1162.49    811.779   568.675  2790.0       1136.34    693.904  1195.26
  880.301   437.075   522.057   614.802      521.774   547.608   946.943
 1462.46    587.197   591.556  1045.19   …   944.425   667.969  1382.24
 1434.56    946.096   826.207  1342.82      1196.96   1049.75   1383.15
 1276.66    651.785   549.673  1361.87      1098.5     580.29   1186.05
 1377.87   1076.05    724.338  1410.39      1376.32   1045.33   1114.97
  869.606   846.611   713.64    655.534      761.013  1116.03    701.958
 1227.59    986.172   816.383  1080.43   …  1069.42   1049.42   1255.51
 1431.85    983.995   647.768  1304.36      1275.42    891.55   1177.11
  688.196   384.223   561.935   566.935      412.38    424.248   835.09
    ⋮                              

Next, compute the matrix product $\mathbf{X}^{\top}\mathbf{X}$ and save it to the `K₂::Array{Float64,2}` variable:

In [24]:
K₂ = transpose(D̄)*D̄ |> Matrix;

Is $\mathbf{K}_{1} = \mathbf{K}_{2}$? We'll use the shortcut version `≈` of [the `isapprox(...)` method](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to check if $\mathbf{K}_{1} = \mathbf{K}_{2}$.
* __Expectation__:  If the kernel function is the inner product, then we expect $\mathbf{K}_{1} = \mathbf{K}_{2}$. Otherwise, we expect $\mathbf{K}_{1} \neq \mathbf{K}_{2}$. If $\mathbf{K}_{1} = \mathbf{K}_{2}$ then this check will return `true`, if not it returns `false`.

In [26]:
K₁ ≈ K₂

true

__Eigenvalues of kernel matrix are non-negative__: A valid kernel function will have a kernel matrix whose eigenvalues are all non-negative. Let the kernel matrix be $\mathbf{K}$ and let $\lambda_{1},\lambda_{2},\dots\lambda_{m}$ be the eigenvalues of $\mathbf{K}$. If $k:\mathbb{R}^{\star}\times\mathbb{R}^{\star}\to\mathbb{R}$ is a valid kernel function, then $\lambda_{i}\geq{0}$ and real $i=1,2\dots,m$.

Let's test this idea using the `K₁::Array{Float64,2}` kernel matrix we computed above. We'll compute the eigendecomposition of the kernel matrix [using the `eigen(...)` method exported by the `LinearAlgebra.jl` package](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen).

In [28]:
λ,V = let

    F = eigen(K₁);
    λ = F.values;
    V = F.vectors;

    λ,V
end;

Are any of the values in the $\lambda$ vector negative?

In [30]:
for i ∈ eachindex(λ)
    λᵢ = λ[i];
    @assert λᵢ ≥ 0.0
end

__Random vector check__: Finally, for any real value vector $\mathbf{x}$, the Kernel matrix $\mathbf{K}$ must satisfy $\mathbf{x}^{\top}\mathbf{K}\mathbf{x} \geq 0$. Let's test this idea using the `K₁::Array{Float64,2}` kernel matrix we computed above.

In [32]:
value = let
    # setup -
    number_of_firms = size(D̄,2); # number of columns in the dataset
    x = randn(number_of_firms); # generate a random vector

    # test -
    value = transpose(x)*K₁*x;

    # return 
    value
end;

In [33]:
println("What is the x^T*K*x value? $(value)")

What is the x^T*K*x value? 1.7592022836452746e6


__Are Kernel Matrices Symmetric__? Let's do a quick to show that are a kernel matrix is symmetric, i.e, that $\mathbf{K}=\mathbf{K}^{\top}$.

In [183]:
transpose(K₁) ≈ K₁

true

In [185]:
transpose(K₂) ≈ K₂

true

## Kernel regression
Kernel regression is a non-parametric technique to model (potentially) non-linear relationships between variables. It moves away from having one _global_ model to describe data and instead uses a weighted combination of many _local_ (potentially) nonlinear models to describe the data.
* __How does it work__? Kernel regression uses a _kernel function_ to assign weights to new (unseen) data points based on their proximity (similarity) to a known data, allowing for estimating a smooth curve or function that describes the relationship between dependent and independent variables. Thus, we shift to a weighted combination of many _local_ models.

### Theory
Suppose we have a dataset $\mathcal{D} = \{(\mathbf{x}_{i},y_{i}) \mid i = 1,2,\dots,n\}$, where the features $\mathbf{x}_i \in \mathbb{R}^{m}$ are $m$-dimensional vectors ($m\ll{n}$) and the target variables are continuous values $y_i \in \mathbb{R}$, e.g., the price of a house, the price of a stock, the temperature, etc. We can model this as a linear regression problem:
$$
\hat{\mathbf{y}} = \hat{\mathbf{X}}\theta
$$
where $\hat{\mathbf{X}}$ is a data matrix with the transpose of the augmented feature vectors $\hat{\mathbf{x}}^{\top}$ on the rows, and $\theta$ is an unknown parameter vector $\theta\in\mathbb{R}^{p}$ where $p = m+1$. The (regularized) least squares solution for the parameters $\theta$ is given by:
$$
\begin{equation}
\hat{\mathbf{\theta}}_{\lambda} = \left(\hat{\mathbf{X}}^{\top}\hat{\mathbf{X}}+\lambda\,\mathbf{I}\right)^{-1}\hat{\mathbf{X}}^{\top}\mathbf{y}
\end{equation}
$$

#### Kernel regression
The basic idea of kernel regression is to rewrite the parameter vector $\hat{\theta}_{\lambda}$ as a sum of the _augmented feature variables_: $\hat{\theta}_{\lambda} \equiv \sum_{i=1}^{n}\alpha_{i}\hat{\mathbf{x}}_{i}$. Then for some (new) feature vector $\hat{\mathbf{z}}$,  the predicted output $\hat{y}$ is given by:
$$
\begin{align}
\hat{y} & = \hat{\mathbf{z}}^{\top}\theta = \sum_{i=1}^{n}\alpha_{i}\left<\hat{\mathbf{z}},\mathbf{x}_{i}\right>\quad\text{|\,Replace inner product with kernel}\\
        & = \hat{\mathbf{z}}^{\top}\theta \simeq \sum_{i=1}^{n}\alpha_{i}\,k(\hat{\mathbf{z}},\mathbf{x}_{i})
\end{align}
$$
where $k(\hat{\mathbf{z}},\mathbf{x}_{i})$ denotes a kernel function (similarity score) between a new (augmented) feature vector and $\hat{\mathbf{z}}$ and the (known) training feature vector $\hat{\mathbf{x}}_{i}$. We need to estimate the $\alpha_{i}$ parameters; however, this is not as hard as it may first appear.

__How are $\alpha$ and $\theta$ related__? 
The two expression for $\hat{\theta}_{\lambda}$ can be equated:
$$
\begin{equation}
\left(\hat{\mathbf{X}}^{\top}\hat{\mathbf{X}}+\lambda\,\mathbf{I}\right)^{-1}\hat{\mathbf{X}}^{\top}\mathbf{y} = \hat{\mathbf{X}}^{\top}\alpha
\end{equation}
$$
After some algebraic manipulation that [is shown in the course notes](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-4/L4a/docs/Notes.pdf), this expression can be solved for the expansion coefficients:
$$
\begin{equation}
\alpha = \left(\mathbf{K}^{\prime}+\lambda\mathbf{I}\right)^{-1}\mathbf{y}
\end{equation}
$$
where $\mathbf{K}^{\prime} = \hat{\mathbf{X}}\hat{\mathbf{X}}^{\top}$, the matrix $\mathbf{I}$ denotes the identity matrix, the vector $\mathbf{y}$ denotes the observed outputs and $\lambda\geq{0}$ denotes the regularization parameter. 

__Hmmm__: That's interesting! In lab tomorrow, we are going to build one of these to model stock prices.

# Today
That's a wrap! What are some things we discussed today?